# Extract CLIP Weights for JAX

Convert PyTorch CLIP (ViT-B/32) to numpy arrays for JAX

## 1. Install Dependencies

In [ ]:
%pip install -q torch torchvision

%pip install -q git+https://github.com/openai/CLIP.git

%pip install -q pillow numpy

## 2. Mount Google Drive

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')

folders = [
    '/content/drive/MyDrive/hope-models/checkpoints',
    '/content/drive/MyDrive/hope-models/exports'
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("Drive mounted")

## 3. Load CLIP Models

Loading ViT-B/32 (base) and ViT-L/14 (large)

In [ ]:
import clip
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")
print("\nLoading CLIP ViT-B/32...")

clip_base, preprocess_base = clip.load("ViT-B/32", device=device)
clip_base.eval()
print("ViT-B/32 loaded")

print("\nLoading CLIP ViT-L/14...")
try:
    clip_large, preprocess_large = clip.load("ViT-L/14", device=device)
    clip_large.eval()
    print("ViT-L/14 loaded")
    has_large = True
except Exception as e:
    print(f"ViT-L/14 not available: {e}")
    has_large = False

for param in clip_base.parameters():
    param.requires_grad = False

if has_large:
    for param in clip_large.parameters():
        param.requires_grad = False

## 4. Extract Weights

Convert PyTorch tensors to numpy

In [ ]:
import numpy as np

def extract_weights(model, name):
    weights = {}
    for param_name, param in model.named_parameters():
        weights[param_name] = param.detach().cpu().numpy()
    print(f"Extracted {len(weights)} layers from {name}")
    return weights

base_weights = extract_weights(clip_base.visual, "ViT-B/32")

if has_large:
    large_weights = extract_weights(clip_large.visual, "ViT-L/14")
else:
    large_weights = None

## 5. CLIP Constants

Normalization parameters and prompts from original Hope

In [ ]:
CLIP_MEAN = np.array([0.48145466, 0.4578275, 0.40821073], dtype=np.float32)
CLIP_STD = np.array([0.26862954, 0.26130258, 0.27577711], dtype=np.float32)
CLIP_INPUT_SIZE = 224

CHAOS_PROMPTS = [
    "completely destroyed corrupted visual data",
    "incomprehensible chaotic noise patterns",
    "severely distorted unrecognizable imagery",
    "broken fragmented visual information",
    "extreme digital corruption and artifacts",
    "meaningless random pixel arrangements",
    "visual chaos with no coherent structure",
    "utterly corrupted incomprehensible forms",
]

NORMAL_PROMPTS = [
    "a clear image",
    "a recognizable picture",
    "coherent visual content",
]

print(f"Input size: {CLIP_INPUT_SIZE}x{CLIP_INPUT_SIZE}")
print(f"Mean: {CLIP_MEAN}")
print(f"Std: {CLIP_STD}")
print(f"Chaos prompts: {len(CHAOS_PROMPTS)}")
print(f"Normal prompts: {len(NORMAL_PROMPTS)}")

## 6. Extract Text Embeddings

Pre-compute embeddings for chaos and normal prompts

In [ ]:
chaos_tokens = clip.tokenize(CHAOS_PROMPTS).to(device)
normal_tokens = clip.tokenize(NORMAL_PROMPTS).to(device)

with torch.no_grad():
    chaos_base = clip_base.encode_text(chaos_tokens)
    chaos_base = chaos_base / chaos_base.norm(dim=-1, keepdim=True)

    normal_base = clip_base.encode_text(normal_tokens)
    normal_base = normal_base / normal_base.norm(dim=-1, keepdim=True)

    if has_large:
        chaos_large = clip_large.encode_text(chaos_tokens)
        chaos_large = chaos_large / chaos_large.norm(dim=-1, keepdim=True)

        normal_large = clip_large.encode_text(normal_tokens)
        normal_large = normal_large / normal_large.norm(dim=-1, keepdim=True)
    else:
        chaos_large = None
        normal_large = None

print(f"Chaos embeddings (base): {chaos_base.shape}")
print(f"Normal embeddings (base): {normal_base.shape}")

if has_large:
    print(f"Chaos embeddings (large): {chaos_large.shape}")
    print(f"Normal embeddings (large): {normal_large.shape}")

## 7. Save to Drive

In [ ]:
import pickle

save_path = '/content/drive/MyDrive/hope-models/checkpoints/clip_data.pkl'

data = {
    'base_weights': base_weights,
    'large_weights': large_weights,
    'chaos_embeddings_base': chaos_base.cpu().numpy(),
    'normal_embeddings_base': normal_base.cpu().numpy(),
    'chaos_embeddings_large': chaos_large.cpu().numpy() if has_large else None,
    'normal_embeddings_large': normal_large.cpu().numpy() if has_large else None,
    'clip_mean': CLIP_MEAN,
    'clip_std': CLIP_STD,
    'clip_input_size': CLIP_INPUT_SIZE,
    'chaos_prompts': CHAOS_PROMPTS,
    'normal_prompts': NORMAL_PROMPTS,
}

with open(save_path, 'wb') as f:
    pickle.dump(data, f)

file_size = os.path.getsize(save_path) / (1024**2)
print(f"Saved to: {save_path}")
print(f"Size: {file_size:.2f} MB")

## 8. Verify Data

In [ ]:
with open(save_path, 'rb') as f:
    loaded = pickle.load(f)

print("Loaded successfully\n")

for key, value in loaded.items():
    if isinstance(value, np.ndarray):
        print(f"{key}: shape {value.shape}")
    elif isinstance(value, dict):
        print(f"{key}: {len(value)} layers")
    elif isinstance(value, list):
        print(f"{key}: {len(value)} items")
    else:
        print(f"{key}: {value}")

## Complete

**Extracted:**
- ViT-B/32 weights
- ViT-L/14 weights (if available)
- Chaos/normal text embeddings
- CLIP constants

**Saved:** `/content/drive/MyDrive/hope-models/checkpoints/clip_data.pkl`

**Next:** Run `2_noise_algorithm.ipynb`